In [2]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent


In [27]:
import openai
import os
import dotenv

env_file = '../../.env'
dotenv.load_dotenv(env_file, override=True)

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_ENDPOINT  = os.environ.get("OPENAI_ENDPOINT")
OPENAI_API_VERSION  = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_TYPE  = os.environ.get("OPENAI_API_TYPE")

openai.api_type = OPENAI_API_TYPE
openai.api_base = OPENAI_ENDPOINT
openai.api_version = OPENAI_API_VERSION
openai.api_key = OPENAI_API_KEY

OPENAI_API_TYPE

'azure'

In [28]:
llm = OpenAI(
    # engine = "asada001",
    # engine="aschatgpt35",
    model_kwargs= {
        "engine": "asdavinci003",
    },
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_ENDPOINT,
    max_tokens=256,
)

# Define Tools

In [29]:
# Import things that are needed generically
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [30]:
# Load the tool configs that are needed.
serpapi_api_key=os.environ.get("SERPAPI_API_KEY")

search = SerpAPIWrapper(serpapi_api_key=serpapi_api_key)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

In [31]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [15]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [16]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [17]:
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)



> Entering new  chain...
 I need to find out the girlfriend's name and age
Action: Search
Action Input: "Leo DiCaprio girlfriend age"

ValueError: Got error from SerpAPI: Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key

In [11]:
agent.run(
    "中国队和缅甸队的比赛结果是什么？进球数的三次方是多少？请用中文回答"
)



> Entering new  chain...
 I should find out the score first
Action: Search
Action Input: 中国队和缅甸队的比赛结果
Observation: 北京时间6月16日18点30分，一场备受中国球迷关注的比赛成功打响，大连梭鱼湾足球场，中国队对阵缅甸队。 最终，国足拿下4-0大胜，武磊6分钟梅开二度！
Thought: I should now calculate the cube
Action: Calculator
Action Input: 4^3

> Entering new  chain...
4^3```text
4**3
```
...numexpr.evaluate("4**3")...

Answer: 64
> Finished chain.

Observation: Answer: 64
Thought: I now know the final answer
Final Answer: 中国队4-0缅甸队，进球数的三次方是64。

> Finished chain.


'中国队4-0缅甸队，进球数的三次方是64。'